# Test Cloud Calc API

Notebook per testare tutte le operazioni esposte da `cloud_calc_api.py`

In [15]:
import requests

BASE_URL = "http://18.153.39.218:5000"

def calc(operation, args):
    """Helper per chiamare l'endpoint /calc via POST"""
    resp = requests.post(f"{BASE_URL}/calc", json={"operation": operation, "args": args})
    print(f"{operation}({', '.join(str(a) for a in args)}) => ", end="")
    if resp.ok:
        data = resp.json()
        print(data["result"])
    else:
        print(f"ERROR {resp.status_code}: {resp.json()}")
    return resp

## Health Check & Operations List

In [16]:
# Health check
resp = requests.get(f"{BASE_URL}/health")
print("Health:", resp.json())

Health: {'status': 'healthy'}


In [17]:
# Lista operazioni disponibili
resp = requests.get(f"{BASE_URL}/operations")
print("Operazioni disponibili:")
for op in resp.json()["operations"]:
    print(f"  - {op}")

Operazioni disponibili:
  - plus
  - minus
  - multiply
  - divide
  - power
  - mod
  - equals
  - greater
  - less
  - greater_equal
  - less_equal
  - and
  - or
  - not
  - if
  - sqrt
  - abs
  - round
  - floor
  - ceil
  - max
  - min
  - average
  - count
  - concat
  - upper
  - lower
  - trim
  - len


## Operazioni matematiche base

In [18]:
calc("plus", [10, 20, 30])       # 60
calc("minus", [100, 37])          # 63
calc("multiply", [3, 4, 5])      # 60
calc("divide", [100, 4])          # 25
calc("divide", [10, 0])           # #DIV/0!
calc("power", [2, 10])            # 1024
calc("mod", [17, 5]);             # 2

plus(10, 20, 30) => 60
minus(100, 37) => 63
multiply(3, 4, 5) => 60
divide(100, 4) => 25.0
divide(10, 0) => #DIV/0!
power(2, 10) => 1024
mod(17, 5) => 2


## Operazioni di confronto

In [21]:
calc("equals", ["ciao", "ciao"])            # True
calc("equals", [5, 3])            # False
calc("greater", [10, 5])          # True
calc("less", [3, 7])              # True
calc("greater_equal", [5, 5])     # True
calc("less_equal", [4, 3]);       # False

equals(ciao, ciao) => True
equals(5, 3) => False
greater(10, 5) => True
less(3, 7) => True
greater_equal(5, 5) => True
less_equal(4, 3) => False


## Operazioni logiche

In [6]:
calc("and", [True, True, True])   # True
calc("and", [True, False, True])  # False
calc("or", [False, False, True])  # True
calc("or", [False, False])        # False
calc("not", [True])               # False
calc("not", [False]);             # True

and(True, True, True) => True
and(True, False, True) => False
or(False, False, True) => True
or(False, False) => False
not(True) => False
not(False) => True


## Operazione condizionale (IF)

In [7]:
calc("if", [True, "SI", "NO"])    # SI
calc("if", [False, "SI", "NO"]); # NO

if(True, SI, NO) => SI
if(False, SI, NO) => NO


## Funzioni matematiche

In [8]:
calc("sqrt", [144])               # 12
calc("abs", [-42])                # 42
calc("round", [3.14159, 2])       # 3.14
calc("round", [3.5])             # 4
calc("floor", [3.9])              # 3
calc("ceil", [3.1]);              # 4

sqrt(144) => 12.0
abs(-42) => 42
round(3.14159, 2) => 3.14
round(3.5) => 4.0
floor(3.9) => 3
ceil(3.1) => 4


## Funzioni aggregate

In [9]:
calc("max", [3, 7, 1, 9, 4])     # 9
calc("min", [3, 7, 1, 9, 4])     # 1
calc("average", [10, 20, 30])     # 20
calc("count", ["a", "b", "c"]);  # 3

max(3, 7, 1, 9, 4) => 9
min(3, 7, 1, 9, 4) => 1
average(10, 20, 30) => 20.0
count(a, b, c) => 3


## Funzioni stringa

In [10]:
calc("concat", ["Hello", " ", "World"])  # Hello World
calc("upper", ["ciao mondo"])              # CIAO MONDO
calc("lower", ["CIAO MONDO"])              # ciao mondo
calc("trim", ["  spazi  "])                # spazi
calc("len", ["test"]);                     # 4

concat(Hello,  , World) => HelloWorld
upper(ciao mondo) => CIAO MONDO
lower(CIAO MONDO) => ciao mondo
trim(  spazi  ) => spazi
len(test) => 4


## Test chiamata GET

In [11]:
# Test con GET (query params)
resp = requests.get(f"{BASE_URL}/calc", params={"operation": "plus", "args": [10, 20]})
print("GET /calc plus(10, 20) =>", resp.json())

GET /calc plus(10, 20) => {'args': [10, 20], 'operation': 'plus', 'result': 30}


## Test errori

In [12]:
# Operazione mancante
resp = requests.post(f"{BASE_URL}/calc", json={"operation": "", "args": [1]})
print("Operazione vuota:", resp.status_code, resp.json())

# Operazione inesistente
resp = requests.post(f"{BASE_URL}/calc", json={"operation": "foobar", "args": [1]})
print("Operazione sconosciuta:", resp.status_code, resp.json())

# Numero argomenti sbagliato
resp = requests.post(f"{BASE_URL}/calc", json={"operation": "minus", "args": [1, 2, 3]})
print("Argomenti sbagliati:", resp.status_code, resp.json())

Operazione vuota: 400 {'error': 'Operation is required'}
Operazione sconosciuta: 400 {'available': ['plus', 'minus', 'multiply', 'divide', 'power', 'mod', 'equals', 'greater', 'less', 'greater_equal', 'less_equal', 'and', 'or', 'not', 'if', 'sqrt', 'abs', 'round', 'floor', 'ceil', 'max', 'min', 'average', 'count', 'concat', 'upper', 'lower', 'trim', 'len'], 'error': 'Unknown operation: foobar'}
Argomenti sbagliati: 400 {'error': 'Invalid number of arguments: <lambda>() takes 2 positional arguments but 3 were given', 'operation': 'minus'}
